In [ ]:
data_lake_account_name = '' # Synapse Workspace ADLS
file_system_name = ''

In [ ]:
inputFolderPath = '2021/11/2/2021-11-02T03:55:08'
initialLoad = True

In [ ]:
if initialLoad == True:
    spark.sql('drop table if exists events_temp')
    spark.sql('drop table if exists eventsdata') 

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
def get_location_flag(Location_DisplayName): 
    location_flag = 'InPerson'
    if 'teams' in Location_DisplayName or 'zoom' in Location_DisplayName \
    or 'webex' in Location_DisplayName or 'loopup' in Location_DisplayName \
    or 'dial' in Location_DisplayName or 'conference' in Location_DisplayName  \
    or '+1' in Location_DisplayName or 'meeting' in Location_DisplayName  \
    or 'cell' in Location_DisplayName or '1-8' in Location_DisplayName  \
    or 'code' in Location_DisplayName or '888' in Location_DisplayName  \
    or '800' in Location_DisplayName or 'call' in Location_DisplayName  \
    or '#' in Location_DisplayName or 'google meet' in Location_DisplayName :
        location_flag = 'Remote'
    return location_flag
    
def load_events_json_file(eventsPath):
    events_df= spark.read.load(eventsPath, format='json')

    location_flag_udf = udf(lambda Location_DisplayName: get_location_flag(Location_DisplayName), returnType=StringType())

    df = events_df.select('*',size('Attendees').alias('Attendees_cnt')) \
                .select(explode(col("Attendees")).alias("Attendees"),"Id","puser","ICalUid","Subject","Recurrence","IsCancelled","Start","End","Organizer","Attendees_cnt","Location") \
                .select(["Id","puser","ICalUid","Subject","Attendees_cnt",
                            when(col("Recurrence").isNull(),False).otherwise(True).alias("Recurrence"),
                            "IsCancelled",
                            col("Start.DateTime").alias("Start"),col("End.DateTime").alias("End"),
                            col("Organizer.EmailAddress.Address").alias("Organizer"),
                            col("Attendees.EmailAddress.Address").alias("Attendee"),
                            col("Attendees.Status.Response").alias("Attendee_Response"),
                            col("Attendees.Type").alias("Attendee_Type"),
                            col("Location.Address.Type").alias("Location_Address_Type"),
                            col("Location.DisplayName").alias("Location_DisplayName")]) \
                .withColumn('location_flag', location_flag_udf(lower(col("Location_DisplayName")))) \
                .withColumn("LoadFolderPath", lit(inputFolderPath))

    df.write.mode("append").saveAsTable("events_temp") 
    
def get_event_subfolder_files(folder):
    children = mssparkutils.fs.ls(folder)
    #print(children[0])
    for child in children:
        if child.name == 'metadata':
            continue
        if child.isDir:
            get_event_subfolder_files(child.path)
        else:
            load_events_json_file(child.path)

In [ ]:
eventsPath = f"abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/o365data/events/"
get_event_subfolder_files(eventsPath)

In [ ]:
sql_str = '''SELECT  Id,puser,ICalUid,Subject,Recurrence,IsCancelled,
to_timestamp(Start) as Start,
to_date(Start) as Start_Date,
to_timestamp(End) as End,
lower(Organizer) as Organizer,lower(Attendee) as Attendee,
Attendee_Response,Attendee_Type,
reverse(split(lower(Organizer),'@'))[0] as Organizer_Domain,
reverse(split(lower(Attendee),'@'))[0] as Attendee_Domain,
Attendees_cnt,location_flag
FROM events_temp'''

sql_str = sql_str + " where LoadFolderPath = '" + inputFolderPath + "'" 

sql_str = sql_str + ' UNION ALL '

sql_str = sql_str + '''SELECT  Id,puser,ICalUid,Subject,Recurrence,IsCancelled,
to_timestamp(Start) as Start,
to_date(Start) as Start_Date,
to_timestamp(End) as End,
lower(Attendee) as Organizer,lower(Organizer) as Attendee,
Attendee_Response,Attendee_Type,
reverse(split(lower(Attendee),'@'))[0] as Organizer_Domain,
reverse(split(lower(Organizer),'@'))[0] as Attendee_Domain,
Attendees_cnt,location_flag
FROM events_temp
'''
sql_str = sql_str + " where LoadFolderPath = '" + inputFolderPath + "'"

eventsdata_df = spark.sql(sql_str)
eventsdata_df.write.mode("append").saveAsTable("eventsdata")